In [1]:
from sklearn.linear_model import *
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import numpy as np
import pandas as pd

In [2]:
data_file = pd.read_csv("../TAMU_FINAL_DATASET_2018.csv")

In [4]:
# only select the numeric data
df = data_file.select_dtypes(include=['float64','int64'])
categorical_data = data_file.select_dtypes(exclude=['float64','int64'])
categorical_df = pd.DataFrame()
for column in categorical_data.columns:
    dummies = pd.get_dummies(categorical_data[column], prefix=column)
    categorical_df = pd.concat([categorical_df, dummies], axis=1)
#df = pd.concat([df['AMI_FLAG'], categorical_df], axis=1)
df = pd.concat([df, categorical_df], axis=1)

In [5]:
sum(df['AMI_FLAG'] == 1)
df_AMI = df[df['AMI_FLAG'] == 1]
df_NOAMI = df[df['AMI_FLAG'] == 0]
df_NOAMI_sample = df_NOAMI.sample(2726)
print(df_AMI.shape)
print(df_NOAMI_sample.shape)

(2726, 469)
(2726, 469)


In [6]:
df = pd.concat([df_AMI, df_NOAMI_sample], axis = 0)
# drop the null value 

print(df.shape)
df = df.dropna()
print(df.shape)

# define the target we would like to predict 
y = df['AMI_FLAG']

X = df.loc[:, df.columns != 'AMI_FLAG']

# sanity check on X, y dimensions 
print(X.shape)
print(y.shape)

# split training and testing data

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state=42)

(5452, 469)
(4062, 469)
(4062, 468)
(4062,)


In [7]:
# construct linear model using sklearn 
lm = LogisticRegression(penalty='l1')
model = lm.fit(X_train, y_train)

pred_train = lm.predict(X_train)
pred_test = lm.predict(X_test)

#print(pred_train[:10])
#print(y_train[:10])

#print(pred_test[:10])
#print(y_test[:10])

In [8]:
# logistic regression result 

print(np.count_nonzero(pred_train))
print(np.count_nonzero(y_train))

print(np.count_nonzero(pred_test))
print(np.count_nonzero(y_test))

print(accuracy_score(y_train, pred_train))
print(accuracy_score(y_test, pred_test))


1210
1309
570
683
0.7578096288129365
0.639821029082774


In [9]:
def eliminate_covariate(df):
    for i in range(len(df.columns)):
        for j in range(len(df.columns)):
            if i != j:
                if np.corrcoef(df.iloc[:,j], df.iloc[:,i])[1,0] > 0.8:
                    print(df.columns[i], df.columns[j])

In [10]:
eliminate_covariate(df)

/anaconda3/lib/python3.6/site-packages/numpy/lib/function_base.py:3183: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/anaconda3/lib/python3.6/site-packages/numpy/lib/function_base.py:3184: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


CON_VISIT_11_Q04 CON_VISIT_11_Q03
CON_VISIT_11_Q04 CON_VISIT_11_Q02
CON_VISIT_11_Q01 CON_VISIT_11_Q03
CON_VISIT_11_Q01 CON_VISIT_11_Q02
CON_VISIT_11_Q03 CON_VISIT_11_Q04
CON_VISIT_11_Q03 CON_VISIT_11_Q01
CON_VISIT_11_Q03 CON_VISIT_11_Q02
CON_VISIT_11_Q02 CON_VISIT_11_Q04
CON_VISIT_11_Q02 CON_VISIT_11_Q01
CON_VISIT_11_Q02 CON_VISIT_11_Q03
CON_VISIT_28_Q03 CON_VISIT_28_Q04
CON_VISIT_16_Q01 CON_VISIT_16_Q02
CON_VISIT_16_Q02 CON_VISIT_16_Q01
CON_VISIT_28_Q04 CON_VISIT_28_Q03
CON_VISIT_12_Q01 POT_VISIT_51_Q01
POT_VISIT_51_Q01 CON_VISIT_12_Q01
POT_VISIT_53_Q02 POT_VISIT_53_Q04
POT_VISIT_53_Q04 POT_VISIT_53_Q02
POT_VISIT_99_Q03 POT_VISIT_99_Q01
POT_VISIT_99_Q03 POT_VISIT_99_Q02
POT_VISIT_99_Q01 POT_VISIT_99_Q03
POT_VISIT_99_Q01 POT_VISIT_99_Q02
POT_VISIT_99_Q02 POT_VISIT_99_Q03
POT_VISIT_99_Q02 POT_VISIT_99_Q01
POT_VISIT_65_Q02 POT_VISIT_65_Q03
POT_VISIT_65_Q02 POT_VISIT_65_Q04
POT_VISIT_65_Q02 POT_VISIT_65_Q01
POT_VISIT_65_Q03 POT_VISIT_65_Q02
POT_VISIT_65_Q03 POT_VISIT_65_Q04
POT_VISIT_65_Q